In [9]:
import csv
from pathlib import Path
import pdfplumber
from tqdm import tqdm

In [10]:
import csv

def listener(q):
    '''listens for messages on the q, writes to file. '''

    with open("/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/emma_file_status.csv", "w+") as f:
        csv_handler = csv.writer(f)
        csv_handler.writerow(["FileName", "Comment", "PageCount"])
        while 1:
            m = q.get()
            if m == 'kill':
                print("Terminated!")
                break
            print(m)
            csv_handler.writerow(m)

# op = open("/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/emma_file_status.csv", "w+")
# csv_handler = csv.writer(op)
# csv_handler.writerow(["FileName", "Comment", "PageCount"])

In [11]:
def worker(fid, q):
    PDF_FILE = f"/OTTO-Project/EMMA_OFFICIAL_STATEMENT_TEMP_PDF/{fid}.pdf"
    res = []
    try:
        if not Path(PDF_FILE).is_file():
            res = [fid, "File doesn't exist", None]
            q.put(res)
            return res
    except Exception as e:
        res = [fid, "OS Exception", None]
        q.put(res)
        return res
    
    try:
        pdf = pdfplumber.open(PDF_FILE)
        res = [fid, None, len(pdf.pages)]
        q.put(res)
        pdf.close()
        return res
    except Exception as e:
        res = [fid, "Corrupted PDF OR File doesn't exist", None]
        q.put(res)
        return res

In [ ]:
import multiprocessing as mp
manager = mp.Manager()
q = manager.Queue()    
pool = mp.Pool(mp.cpu_count() + 2)

#put listener to work first
watcher = pool.apply_async(listener, (q,))

#fire off workers
jobs = []
with open("/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/emma_all.csv", encoding="utf8") as f:
    for fid in tqdm(f):
        fid = fid.upper().strip()
        job = pool.apply_async(worker, (fid, q))
        jobs.append(job)

# collect results from the workers through the pool result queue
for job in jobs: 
    job.get()

#now we are done, kill the listener
q.put('kill')
pool.close()
pool.join()

In [ ]:
# with open("/home/factentry/otto_ml/src/antlr/mondal/pdf_table_extractor/emma_all.csv", encoding="utf8") as f:
#     for fid in tqdm(f):
#         fid = fid.upper().strip()
#         PDF_FILE = f"/OTTO-Project/EMMA_OFFICIAL_STATEMENT_TEMP_PDF/{fid}.pdf"
#         try:
#             if not Path(PDF_FILE).is_file():
#                 csv_handler.writerow([fid, "File doesn't exist", None])
#         except Exception as e:
#             csv_handler.writerow([fid, "OS Exception", None])
        
#         try:
#             pdf = pdfplumber.open(PDF_FILE)
#             csv_handler.writerow([fid, None, len(pdf.pages)])
#         except Exception as e:
#             csv_handler.writerow([fid, "Corrupted PDF", None])
#         finally:
#             pdf.close()


748it [08:41,  5.87it/s]

In [ ]:
# op.close()